# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
2. Công cụ sử dụng
3. Các bước thu thập dữ liệu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install requests_html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.6 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=501df2cab3613835b023344f3e8d49930a4b9f583d02200519f1de7df263fb99
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [9]:
# import những  thư viện cần thiết
import pandas as pd
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import math
import requests
import time

## Hàm gửi yêu cầu đến server yêu cầu tải toàn bộ nội dung trang web

In [10]:

def get_content(url, headers):
    # time.sleep(1)
    session = HTMLSession()
    response = session.get(url, headers=headers)
    if(response.status_code == 200):
        print("Success!!!")
        return response
    return None

In [11]:
BASE_URL = 'https://vieclam24h.vn'


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [17]:
# Code demo request lấy page list công việc
job_pool = pd.DataFrame()
for work_method in range(1,7):
   #https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6
   url = BASE_URL + '/tim-kiem-viec-lam-nhanh?&working_method={}'.format(work_method)
   print(url)
   page_content = get_content(url, headers)
   soup = BeautifulSoup(page_content.text, 'html.parser')

   buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
   buffer2 = buffer1.find('span',class_="font-semibold").text
   n_jobs = int(buffer2.replace(',',''))
   # Trung bình mỗi page có 30 công việc
   n_pages = min(51, math.ceil(n_jobs/30) + 1)
   print('Có {} công việc, {} trang ở work_method = {}'.format(n_jobs, n_pages, work_method))


   for page in range(1,n_pages):
      # Khởi tạo url vầ yêu cầu lấy page về
      url = BASE_URL + '/tim-kiem-viec-lam-nhanh?page={}&working_method={}'.format(page, work_method)
      print(url)
      page_content = get_content(url, headers)
      soup = BeautifulSoup(page_content.text, 'html.parser')

      containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
      print('Đã tìm thấy {} công việc'.format(len(containers)))
      for container in containers:
         dict_job = {}

         buffer = BASE_URL + container.find('a', href=True)['href']
         dict_job['Liên kết'] = buffer if buffer else None

         buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
         dict_job['Tên công việc'] = buffer.text if buffer else None

         buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
         dict_job['Tên công ty'] = buffer.text if buffer else None

         buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
         dict_job['Mức lương'] = buffer.text if buffer else None

         buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
         dict_job['Khu vực tuyển'] = buffer.text if buffer else None

         job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool.sample(5)



https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=1
Success!!!
Có 5717 công việc, 51 trang ở work_method = 1
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=3&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=4&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=5&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=6&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=7&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=8&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
1837,https://vieclam24h.vn/ban-hang-kinh-doanh/thuc...,Thực Tập Sinh Kinh Doanh,Công Ty TNHH Cơ Khí Việt,5 - 10 triệu,TP.HCM
1408,https://vieclam24h.vn/ban-hang-kinh-doanh/pho-...,Phó Phòng Kinh Doanh Thiết Bị Điện,Công Ty Cổ Phần Thiết Bị Điện Vi-Na-Si-No,13 - 17 triệu,TP.HCM
1520,https://vieclam24h.vn/hanh-chinh-thu-ky/nhan-v...,Nhân Viên Tổ Chức Sự Kiện - Event Executive - ...,Abbott Việt Nam,10 - 15 triệu,TP.HCM
450,https://vieclam24h.vn/ban-hang-kinh-doanh/viet...,Vietinbank Hà Nội / Hưng Yên / Quảng Trị - Chu...,Công ty TNHH Manulife ( Việt Nam ),10 - 20 triệu,"Hà Nội, Thái Nguyên, Lào Cai, Hưng Yên, +1"
1141,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,Nhân Viên Tư Vấn Tín Dụng Kênh Trả Góp Xe Máy ...,Công ty Tài Chính TNHH HD SAISON,8 - 15 triệu,Cần Thơ


In [21]:
# Lưu dữ liệu vào file csv
job_pool.to_csv('../data/job_pool.csv', encoding='utf-8-sig', index=False)

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [22]:
# Đọc dữ liệu từ file job_links.csv
df = pd.read_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/job_pool.csv')
df.sample(5)

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
1141,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,Nhân Viên Tư Vấn Tín Dụng Kênh Trả Góp Xe Máy ...,Công ty Tài Chính TNHH HD SAISON,8 - 15 triệu,Cần Thơ
1129,https://vieclam24h.vn/hanh-chinh-thu-ky/tro-ly...,Trợ Lý Kinh Doanh,Công Ty TNHH TM Dịch Vụ Bảo Vệ An Tâm Sg,9 - 15 triệu,"Long An, Bình Dương, TP.HCM"
1151,https://vieclam24h.vn/xay-dung/ky-su-du-toan-c...,Kỹ Sư Dự Toán,Công Ty CP Tư Vấn - Đầu Tư - Thiết Kế - Xây Dự...,15 - 25 triệu,TP.HCM
1486,https://vieclam24h.vn/marketing/nhan-vien-kinh...,Nhân Viên Kinh Doanh,Công Ty TNHH Sản Xuất Thịnh Việt,20 - 35 triệu,Bình Dương
1372,https://vieclam24h.vn/ban-hang-kinh-doanh/truo...,Trưởng Phòng Kinh Doanh Thiết Bị Công Nghiệp,Công Ty Cổ Phẩn Secons,Thoả thuận,TP.HCM


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1910 entries, 0 to 1909
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Liên kết       1910 non-null   object
 1   Tên công việc  1910 non-null   object
 2   Tên công ty    1910 non-null   object
 3   Mức lương      1910 non-null   object
 4   Khu vực tuyển  1910 non-null   object
dtypes: object(5)
memory usage: 74.7+ KB


In [27]:
raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])
for i, url in enumerate(df['Liên kết']):
    print("Công việc thứ {}".format(i))
    job_content = get_content(url, headers=headers)
    if (job_content.status_code == 200):
      soup = BeautifulSoup(job_content.text, 'html.parser')

      dict_job = {}

      dict_job['Liên kết'] = url

      buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
      dict_job['Tên công ty'] = buffer.text if buffer else None

      buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
      dict_job['Tên công việc'] = buffer.text if buffer else None

      buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
      dict_job['Mức lương'] = buffer.text if buffer else None

      buffer = soup.find('a', class_='hover:text-se-accent')
      dict_job['Khu vực tuyển'] = buffer.text if buffer else None


      buffer = soup.find('p', string="Cấp bậc")
      dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

      buffer = soup.find('p', string="Số lượng tuyển")
      dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

      buffer = soup.find('p', string="Hình thức làm việc")
      dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

      buffer = soup.find('p', string="Yêu cầu bằng cấp")
      dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

      buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
      dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

      buffer = soup.find('p', string="Ngành nghề")
      dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


      buffer1 = soup.find('h4', string="Mô tả công việc")
      buffer2 = buffer1.find_next_sibling('div')
      dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

      buffer1 = soup.find('h4', string="Yêu cầu công việc")
      buffer2 = buffer1.find_next_sibling('div')
      dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

      buffer1 = soup.find('h4', string="Quyền lợi")
      buffer2 = buffer1.find_next_sibling('div')
      dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

      buffer1 = soup.find('h4', string="Địa điểm làm việc")
      buffer2 = buffer1.find_next_sibling('ul')
      dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None


      buffer1 = soup.find('h3', string="Từ khoá")
      buffer2 = buffer1.find_next_sibling('div')
      dict_job['Từ khóa'] = buffer2.text if buffer2 else None

      company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

      buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
      for buffer in buffers:
          if buffer.get_text() == 'Địa chỉ:':
              dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
          if buffer.get_text() == 'Quy mô:':
              dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

      raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

Công việc thứ 0
Success!!!
Công việc thứ 1
Success!!!
Công việc thứ 2
Success!!!
Công việc thứ 3
Success!!!
Công việc thứ 4
Success!!!
Công việc thứ 5
Success!!!
Công việc thứ 6
Success!!!
Công việc thứ 7
Success!!!
Công việc thứ 8
Success!!!
Công việc thứ 9
Success!!!
Công việc thứ 10
Success!!!
Công việc thứ 11
Success!!!
Công việc thứ 12
Success!!!
Công việc thứ 13
Success!!!
Công việc thứ 14
Success!!!
Công việc thứ 15
Success!!!
Công việc thứ 16
Success!!!
Công việc thứ 17
Success!!!
Công việc thứ 18
Success!!!
Công việc thứ 19
Success!!!
Công việc thứ 20
Success!!!
Công việc thứ 21
Success!!!
Công việc thứ 22
Success!!!
Công việc thứ 23
Success!!!
Công việc thứ 24
Success!!!
Công việc thứ 25
Success!!!
Công việc thứ 26
Success!!!
Công việc thứ 27
Success!!!
Công việc thứ 28
Success!!!
Công việc thứ 29
Success!!!
Công việc thứ 30
Success!!!
Công việc thứ 31
Success!!!
Công việc thứ 32
Success!!!
Công việc thứ 33
Success!!!
Công việc thứ 34
Success!!!
Công việc thứ 35
Success!!!
Cô

In [28]:
raw_job_details.sample(5)

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
681,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Công ty Tài Chính TNHH HD SAISON,Nhân Viên Tư Vấn Tín Dụng Kênh Trả Góp - Quận ...,8 - 15 triệu,TP.HCM,Chuyên viên- nhân viên,5,Toàn thời gian cố định,Trung học,Chưa có kinh nghiệm,Bán hàng - Kinh doanh/Ngân hàng/Chăm sóc khách...,"TP.HCM, Quận 11TP.HCM, Quận 1TP.HCM, Thủ ĐứcTP...",- Khi trở thành Nhân viên Điểm giới thiệu dịch...,- Bạn chỉ cần tốt nghiệp trình độ từ THPT trở ...,"- Làm việc trong một môi trường chuyên nghiệp,...",hcmtín dụngquận 11Việc làm TP.HCMBán hàng - Ki...,"Tầng 8, Tòa nhà Gilimex, 24C Phan Đăng Lưu, ph...",Trên 300 người
585,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,Công ty Tài Chính TNHH HD SAISON,Nhân Viên Tư Vấn Tín Dụng - Kênh Trả Góp Điện ...,8 - 15 triệu,Hà Nội,Chuyên viên- nhân viên,10,Toàn thời gian cố định,Trung học,Dưới 1 năm,Tài chính - Đầu tư - Chứng Khoán/Ngân hàng/Bán...,"Hà Nội, Hà Đông, Hà Đông",- Khi trở thành Nhân viên Điểm giới thiệu dịch...,- Bạn chỉ cần tốt nghiệp trình độ từ THPT trở ...,"- Làm việc trong một môi trường chuyên nghiệp,...",Việc làm Hà NộiTài chính - Đầu tư - Chứng Khoá...,"Tầng 8, Tòa nhà Gilimex, 24C Phan Đăng Lưu, ph...",Trên 300 người
1180,https://vieclam24h.vn/xay-dung/can-bo-ky-thuat...,Công Ty TNHH Tư Vấn Xây Dựng Phú Vĩnh Thành,Cán Bộ Kỹ Thuật Hiện Trường,15 - 20 triệu,Bình Dương,Chuyên viên- nhân viên,2,Toàn thời gian cố định,Đại học,3 năm,Xây dựng/Quản lý dự án/Khoa học - Kỹ thuật,"Bình Dương, 47 Đông Minh, Kp Tây A, Đông Hòa, ...","- Trực tiếp quản lý, điều hành và triển khai t...",- Có 3 - 5 năm kinh nghiệm thi công cầu đường ...,- Lương 15 - 20 triệu/tháng- Thưởng lễ tết the...,hiện trườngkỹ thuật hiện trườnghuyện dĩ anViệc...,"Số 47, Đường Đông Minh, Khu phố Tây A - Phường...",Dưới 10 người
409,https://vieclam24h.vn/xuat-nhap-khau/nhan-vien...,Công Ty TNHH Thương Mại Fan VN,Nhân Viên Xuất Nhập Khẩu (Khai Báo Hải Quan),10 - 15 triệu,Đồng Nai,Chuyên viên- nhân viên,1,Toàn thời gian cố định,Cao đẳng,3 năm,Xuất Nhập Khẩu/Kế toán,"Đồng Nai, Số 173 Đường số 3, KDC Hố Nai, Trảng...","Nhận thông tin từ bộ phận kinh doanh, nghiệp v...","- Giới tính: nữ; HK: ưu tiên Biên Hoà, Đồng Na...",- Mức lương: Thỏa thuận theo năng lực- Chế độ ...,nhậpxuấtxuất nhậpViệc làm Đồng NaiXuất Nhập Kh...,"173 Đường số 3, KDC Hố Nai, Hố Nai 3, Trảng B...",10 - 150 người
1192,https://vieclam24h.vn/xuat-ban-in-an/nhan-vien...,Công Ty TNHH Thiết Kế Và In Ấn Việt Mỹ,Nhân Viên Thiết Kế In Ấn Tại Tp.HCM,8 - 11 triệu,TP.HCM,Chuyên viên- nhân viên,1,Toàn thời gian cố định,Trung cấp,2 năm,Xuất bản - In ấn/Thiết kế - Sáng tạo nghệ thuật,"TP.HCM, 38/7 Nguyễn Văn Vịnh, Phường Hiệp Tân,...",- Thiết kế thẻ nhựa - thẻ cào - card visit - t...,- Sử dụng phần mềm: Corel - Ai - PTS là chủ yế...,- Lương + Thưởng thỏa thuận theo năng lực từ 8...,"inhcm Golang,NodeJS,Việc làm TP.HCMXuất bản - ...","38/7 Nguyễn Văn Vịnh, Phường Hiệp Tân, Quận Tâ...",10 - 150 người


In [29]:
# Lưu vào file craw_dataset.csv
raw_job_details.info()
raw_job_details.to_csv('../data/raw_dataset.csv', encoding='utf-8-sig', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1910 entries, 0 to 1909
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Liên kết             1910 non-null   object
 1   Tên công ty          1910 non-null   object
 2   Tên công việc        1910 non-null   object
 3   Mức lương            1910 non-null   object
 4   Khu vực tuyển        1910 non-null   object
 5   Cấp bậc              1910 non-null   object
 6   Số lượng tuyển       1910 non-null   object
 7   Hình thức làm việc   1910 non-null   object
 8   Yêu cầu bằng cấp     1681 non-null   object
 9   Yêu cầu kinh nghiệm  1910 non-null   object
 10  Ngành nghề           1910 non-null   object
 11  Địa điểm làm việc    1910 non-null   object
 12  Mô tả công việc      1910 non-null   object
 13  Yêu cầu công việc    1910 non-null   object
 14  Quyền lợi            1910 non-null   object
 15  Từ khóa              1910 non-null   object
 16  Địa ch

# Một số phần demo để testing
- Đây là đoạn code để người collect data có thể test thử trước khi chạy toàn bộ code

In [ ]:
# Demo lất page content
with open('../data/page_content.txt', 'r', encoding='utf-8') as f:
    page_text = f.read()

soup = BeautifulSoup(page_text, 'html.parser')
buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
buffer2 = buffer1.find('span',class_="font-semibold").text
n_jobs = int(buffer2.replace(',',''))
n_pages = min(50, math.floor(n_jobs/30))
n_pages

50

In [ ]:
# Demo lất page content
with open('../data/page_content.txt', 'r', encoding='utf-8') as f:
    page_text = f.read()

job_pool = pd.DataFrame(columns=['Liên kết', 'Tên công việc','Tên công ty','Mức lương','Khu vực tuyển'])

soup = BeautifulSoup(page_text, 'html.parser')
containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
print('Đã tìm thấy {} công việc'.format(len(containers)))
for container in containers:
    dict_job = {}

    buffer = BASE_URL + container.find('a', href=True)['href']
    dict_job['Liên kết'] = buffer if buffer else None

    buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None

    job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool

Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
0,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên CSKH - Ứng Dụng Du Lịch Tiếng Anh (Q...,Công Ty TNHH Vietnam Concentrix Services,10 - 15 triệu,TP.HCM
1,https://vieclam24h.vn/hanh-chinh-thu-ky/bien-p...,Biên Phiên Dịch Tiếng Trung Tại Long An,Công Ty Cổ Phần Gia Phú,10 - 20 triệu,Long An
2,https://vieclam24h.vn/ban-hang-kinh-doanh/tro-...,Trợ Lý Phát Triển Kinh Doanh,Công Ty Cổ Phần Giấy CP,12 - 14 triệu,TP.HCM
3,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Telesales (Chợ Tốt Xe),Công Ty TNHH Chợ Tốt Trading,6 - 10 triệu,Hà Nội
4,https://vieclam24h.vn/cham-soc-khach-hang/nhan...,Nhân Viên Dịch Vụ Khách Hàng,Mcredit - Công Ty Tài Chính TNHH Mb Shinsei,10 - 20 triệu,Hà Nội
5,https://vieclam24h.vn/ban-hang-kinh-doanh/chuy...,Chuyên Viên Tư Vấn Tài Chính Hồ Chí Minh (Lươn...,Công Ty TNHH Bảo Hiểm Nhân Thọ AIA Việt Nam,16 - 30 triệu,TP.HCM
6,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Kinh Doanh Nội Thất - Thu Nhập 20-30...,Công Ty TNHH Kiến Trúc Milan,20 - 30 triệu,Hà Nội
7,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Trợ Lý (Biết Tiếng Trung),Công Ty TNHH Việt Nam Paiho,7 - 15 triệu,TP.HCM
8,https://vieclam24h.vn/ke-toan/ke-toan-truong-c...,Kế Toán Trưởng,Công Ty TNHH Thương Mại Dịch Vụ Phát Triển Sản...,18 - 30 triệu,TP.HCM
9,https://vieclam24h.vn/ngan-hang/hcm-giam-doc-q...,HCM - Giám Đốc Quan Hệ Khách Hàng Ưu Tiên,Acb - Ngân Hàng Thương Mại Cổ Phần Á Châu,16 - 50 triệu,TP.HCM


In [ ]:

# job = get_content('https://vieclam24h.vn/hanh-chinh-thu-ky/bien-phien-dich-tieng-trung-tai-long-an-c1p123id200290693.html',headers= headers)

# with open('../data/job_content.txt', 'w', encoding='utf-8') as f:
#     f.write(str(job.text))

In [ ]:
# Demo lấy chi tiết

raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Cấp bậc', 'Số lượng tuyển', 'Hình thức làm việc', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

# with open('../data/job_content.txt', 'r', encoding='utf-8') as f:
#     job_content = f.read()
url = 'https://vieclam24h.vn/xay-dung/chi-huy-truong-cong-trinh-c31p73id200288561.html'
job_content = get_content(url, headers=headers).text
soup = BeautifulSoup(job_content, 'html.parser')

dict_job = {}

dict_job['Liên kết'] = 'https://vieclam24h.vn/ban-hang-kinh-doanh/'

buffer = soup.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
dict_job['Tên công ty'] = buffer.text if buffer else None

buffer = soup.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
dict_job['Tên công việc'] = buffer.text if buffer else None

buffer = soup.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
dict_job['Mức lương'] = buffer.text if buffer else None

buffer = soup.find('a', class_='hover:text-se-accent')
dict_job['Khu vực tuyển'] = buffer.text if buffer else None


buffer = soup.find('p', text="Cấp bậc")
dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Số lượng tuyển")
dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Hình thức làm việc")
dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu bằng cấp")
dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Yêu cầu kinh nghiệm")
dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None

buffer = soup.find('p', text="Ngành nghề")
dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None


buffer1 = soup.find('h4', text="Mô tả công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Yêu cầu công việc")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Quyền lợi")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Quyền lợi'] = buffer2.text if buffer2 else None

buffer1 = soup.find('h4', text="Địa điểm làm việc")
buffer2 = buffer1.find_next_sibling('ul')
dict_job['Địa điểm làm việc'] = buffer2.text if buffer2 else None


buffer1 = soup.find('h3', text="Từ khoá")
buffer2 = buffer1.find_next_sibling('div')
dict_job['Từ khóa'] = buffer2.text if buffer2 else None

company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
for buffer in buffers:
    if buffer.get_text() == 'Địa chỉ:':
        dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
    if buffer.get_text() == 'Quy mô:':
        dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None

raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
raw_job_details


Success!!!


C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Cấp bậc")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Số lượng tuyển")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Hình thức làm việc")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  buffer = soup.find('p', text="Yêu cầu bằng cấp")
C:\Users\huynh\AppData\Local\Temp\ipykernel_7408\539015741.py:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecate

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Cấp bậc,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
0,https://vieclam24h.vn/ban-hang-kinh-doanh/,Công Ty CP Hóa Chất Xây Dựng Bách Khoa,Chỉ Huy Trưởng Công Trình,13 - 17 triệu,Hà Nội,Quản lý cấp trung,2,Toàn thời gian cố định,Đại học,2 năm,Xây dựng/Quản lý dự án,"Hà Nội, Phòng 2904, tháp A, chung cư thăng lon...",- Điều phối chung các công việc tại dự án.- Là...,- Tốt nghiệp đại học- Có chứng chỉ giám sát hi...,- Lương: Cạnh tranh theo năng lực (Thu nhập : ...,công trìnhchỉ huy trưởng công trình tại hà nội...,"Phòng 2904, tháp A, chung cư thăng long Number...",10 - 150 người
